In [1]:
from pymongo import MongoClient
from collections import deque
from metadataclient.api import (insert_run_start, insert_descriptor,
                                insert_run_stop, insert_event, bulk_insert_events)
from metadataclient.api import (find_run_starts, find_descriptors, find_events,
                               find_run_stops, find_last)
from metadataclient.conf import connection_config
from urllib3.exceptions import ConnectionError
from metadataclient.api import (insert_run_start, insert_descriptor, insert_event,
                               bulk_insert_events, insert_run_stop)

In [2]:
class MdsMigrationException(RuntimeError):
    """Goodbye cruel world!"""
    pass

# Before this script is run, make sure you create a database called metadataservice

# TODO: Switch to deque from OrderedDict if too slow

**_Ensure global connection management is functional by providing fake host information_**

In [3]:
try:
    MONGO_HOST = 'kusadasi'
    connection_config['host'] = MONGO_HOST 
    res = next(find_last())
except Exception:
    pass

**Connection Management**

In [4]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
MIGRATION_DB = 'datastore2'
# assuming both mongo and tornado server on the same machine
connection_config['host'] = MONGO_HOST 
connection_config['port'] = 7770
connection_config['database'] = 'metadataservice'
pymongo_client = MongoClient(MONGO_HOST, MONGO_PORT)
database = pymongo_client[MIGRATION_DB]
# try:
#     res = (next(find_last()))
# except Exception:
#     raise MdsMigrationException('Seems like metadataservice is not accessible or empty')

**Get all run_starts**

In [5]:
print(database) # visual sanity check


Database(MongoClient('localhost', 27017), 'datastore2')


In [6]:
def rstart_oid_lookup(database, oid):
    try:
        res = next(database.run_start.find({'_id':oid}))
    except StopIteration:
        raise MdsMigrationException('Seems like run_start with this oid does not exist')
    return res

In [7]:
def descriptor_oid_lookup(database, oid):
    try:
        res = next(database.event_descriptor.find({'_id':oid}))
    except StopIteration:
        raise MdsMigrationException('Seems like descriptor with this oid does not exist')
    return res

**Save all run_starts into an OrderedDict()**

**Go over all beamline configurations, delete run_start reference fields, add as a embedded document to run_start, and update entry in run_starts deque()**

In [8]:
rstart_crsr = database.run_start.find()
run_starts = deque()
for rstart in rstart_crsr:
    rs_id = rstart.pop('_id') # no leakeage from older version!
    bcfg = database.beamline_config.find({'run_start': rs_id})
    configs = {}
    for b in bcfg:
        del(b['_id'], b['run_start'])
        configs[b['uid']] = b
    params = dict(time=rstart.pop('time'), scan_id=rstart.pop('scan_id'),
                 beamline_id=rstart.pop('beamline_id'), uid=rstart.pop('uid'), 
                 config= (configs if configs else {}), migrated=True,
                 **rstart)
    run_starts.append(rstart)
    insert_run_start(**params)

HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:7770/run_start

#### **Get all descriptors and insert them using metadataclient.Iterate over descriptors, replace run_start oid references with run_start uid foreign keys, and save them to a deque and a dict that is keyed on uids(for caching)**

In [9]:
# TODO: Test this with an older schema
desc_crsr = database.event_descriptor.find()
descriptors = deque()
desc_dict = {}
for desc in desc_crsr:
    rstart = rstart_oid_lookup(database, desc['run_start'])
    # make sure run_start exists
    del(desc['_id']) # clear the leaked oid field 
    desc['run_start'] = rstart['uid'] # overwrite the old foreign key
    desc['migrated'] = True
    desc_dict[desc['uid']] = desc # keep reference for the events
    insert_descriptor(**desc_dict)
    descriptors.append(desc)

MdsMigrationException: Seems like run_start with this oid does not exist

**Insert all run_starts and descriptors so far**

In [10]:
# pop descriptor from deque, get all corresponding events
# bulk insert events at hand
for d in descriptors:
    events = deque()
    event_crsr = database.event_descriptor.find({'descriptor': d['_id']})
    for e in event_crsr:
        e['descriptor'] = d['uid']
        events.append(e)
    bulk_insert_events(event_descriptor=d, events=events)

In [13]:
rstop_crsr = database.run_stop.find()
rstops = deque()
for rstop in rstop_crsr:
    del(rstop['_id'])
    rstart = rstart_oid_lookup(database, rstop['run_start'])
    rstop['run_start'] = rstart['uid']
    insert_run_stop(migrated=True, **rstop)
    

MdsMigrationException: Seems like run_start with this oid does not exist